In [1]:
import numpy as np
from scipy.io import savemat
import matplotlib.pyplot as plt
from QG_Diagnostics_topo import qg_diagnostics_topo
from Spectrum_topo import spectrum_topo
from RHS_Spectral_topo import rhs_spectral_topo
import time as time


In [3]:
# Set simulation parameters
N = 256       # Number of points in each direction
dt = 5E-4     # initial time step size
Nt = int(1E5) # Number of time steps
qlim = 1.5E5  # if any q > qlim, simulation stops

# Set physical parameters
kd = 10       # Nondimensional deformation wavenumber
kb = 0        # Nondimensional beta wavenumber, beta = kb^2 
U = 1         # zonal shear flow
r = 8         # Nondimensional Ekman friction coefficient
nu = 5E-15    # Coefficient of biharmonic vorticity diffusion
H = 0         # Topography parameter

# Set up hyperviscous PV dissipation
k = np.concatenate((np.arange(0, N//2 + 1), np.arange(-N//2 + 1, 0)))
L = np.zeros((N, N, 2))
for jj in range(N):
    for ii in range(N):
        kr = np.sqrt(k[ii]**2 + k[jj]**2)
        L[ii, jj, :] = -nu * kr**8

# KX, KY = np.meshgrid(k, k)
# KR = np.sqrt(KX**2 + KY**2)
# L = -nu * KR**8

# Initialize topography
dx = 2 * np.pi / N
X, Y = np.meshgrid(np.arange(-np.pi, np.pi, dx), np.arange(-np.pi, np.pi, dx))
topo = H * (np.cos(X) + 2 * np.cos(2 * X))
topo -= np.mean(topo)  # subtracting the mean to center the topography

# Compute the Fourier transform of the topography
hk = np.fft.fft2(topo)

params = {
    'U': U,
    'kd': kd,
    'kb': kb,
    'r': r,
    'nu': nu,
    'N': N,
    'dt': dt,
    'hk': hk
}

# Initialize potential vorticity
qp = np.zeros((N, N, 2))
qp[:, :, 1] = 10 * np.random.randn(N, N)
qp[:, :, 1] -= np.mean(qp[:, :, 1])  # Centering the PV
qp[:, :, 0] = qp[:, :, 1]
q = np.fft.fft2(qp)

# Initialize additional variables for the simulation
xx, yy = np.meshgrid(np.linspace(-np.pi, np.pi, N), np.linspace(-np.pi, np.pi, N))
Ut = U  # zonal shear flow

# Diagnostics (initializing arrays to store diagnostic information)
countDiag = 100  # Compute diagnostics every countDiag steps
T = np.zeros(Nt // countDiag)
vb = np.zeros(Nt // countDiag)       # flux transport
utz = np.zeros((N, Nt // countDiag)) # zonal mean flow
ke = np.zeros((N//2 + 1, Nt // countDiag))  # kinetic energy
ape = np.zeros((N//2 + 1, Nt // countDiag)) # available potential energy

# adaptive stepping stuff:
tol = 1E-1
r0 = .8 * tol

# Main loop
t = 0
start_time = time.time()
for ii in range(1, Nt + 1):
    if ii % countDiag == 0:
        if np.any(np.isnan(q)):
            break
        T[ii // countDiag] = t
        KE, APE = spectrum_topo(q, params)
        ke[:, ii // countDiag] = KE
        ape[:, ii // countDiag] = APE
        VB, UTZ, ene, etp = qg_diagnostics_topo(q, params)
        vb[ii // countDiag] = VB
        utz[:, ii // countDiag] = UTZ

        print(f'iteration i = {ii}; time step dt = {dt}, ene = {np.sum(KE + APE)}')
        print(time.time() - start_time)

    M = 1 / (1 - .25 * dt * L)
    # First stage ARK4
    k0, psik = rhs_spectral_topo(q, params, Ut)
    l0 = L * q
    # Second stage
    q1 = M * (q + .5 * dt * k0 + .25 * dt * l0)
    k1, psik = rhs_spectral_topo(q1, params, Ut)
    l1 = L * q1
    # Third stage
    q2 = M * (q + dt * (13861 * k0 / 62500 + 6889 * k1 / 62500 + 8611 * l0 / 62500 - 1743 * l1 / 31250))
    k2, psik = rhs_spectral_topo(q2, params, Ut)
    l2 = L * q2
    # Fourth stage
    q3 = M * (q + dt * (-0.04884659515311858 * k0 - 0.1777206523264010 * k1 + 0.8465672474795196 * k2 +
                        0.1446368660269822 * l0 - 0.2239319076133447 * l1 + 0.4492950415863626 * l2))
    k3, psik = rhs_spectral_topo(q3, params, Ut)
    l3 = L * q3
    # Fifth stage
    q4 = M * (q + dt * (-0.1554168584249155 * k0 - 0.3567050098221991 * k1 + 1.058725879868443 * k2 +
                        0.3033959883786719 * k3 + 0.09825878328356477 * l0 - 0.5915442428196704 * l1 +
                        0.8101210538282996 * l2 + 0.2831644057078060 * l3))
    k4, psik = rhs_spectral_topo(q4, params, Ut)
    l4 = L * q4
    # Sixth stage
    q5 = M * (q + dt * (0.2014243506726763 * k0 + 0.008742057842904184 * k1 + 0.1599399570716811 * k2 +
                        0.4038290605220775 * k3 + 0.2260645738906608 * k4 + 0.1579162951616714 * l0 +
                        0.1867589405240008 * l2 + 0.6805652953093346 * l3 - 0.2752405309950067 * l4))
    k5, psik = rhs_spectral_topo(q5, params, Ut)
    l5 = L * q5
        # Error control
    r1 = dt * np.max(np.abs(np.fft.ifft2(0.003204494398459 * (k0 + l0) -
                                         0.002446251136679 * (k2 + l2) -
                                         0.021480075919587 * (k3 + l3) +
                                         0.043946868068572 * (k4 + l4) -
                                         0.023225035410765 * (k5 + l5))))
    if r1 > tol:
        dt *= 0.75
        continue

    # Successful step, proceed to evaluation
    t += dt
    qp = np.real(np.fft.ifft2(q + dt * (0.1579162951616714 * (k0 + l0) +
                                        0.1867589405240008 * (k2 + l2) +
                                        0.6805652953093346 * (k3 + l3) -
                                        0.2752405309950067 * (k4 + l4) +
                                        (k5 + l5) / 4)))
    q = np.fft.fft2(qp)

    # Step size adjustment
    dt *= ((0.75 * tol / r1) ** (0.3 / 4)) * ((r0 / r1) ** (0.4 / 4))
    r0 = r1

    if np.any(np.abs(qp) > qlim):
        print(f'qp = {np.max(np.abs(qp))}')
        break

end_time = time.time()
print(f"Total time elapsed: {end_time - start_time} seconds")

# Check for NaNs in q and save variables to .mat file
if np.any(np.isnan(q)):
    print('NaN')
else:
    savemat('QG_DATA_latest.mat', {'ii': ii, 'countDiag': countDiag, 'dt': dt, 'tol': tol, 'params': params, 
                                   'T': T, 'ke': ke, 'ape': ape, 'vb': vb, 'utz': utz, 'qp': qp})



/Users/ree/Documents/Research/Proj_1_LangrangeDA/code/qg_2layer_topo/QG_Diagnostics_topo.py:17: RuntimeWarning: divide by zero encountered in divide
  InvBT = 1 / Laplacian
/Users/ree/Documents/Research/Proj_1_LangrangeDA/code/qg_2layer_topo/QG_Diagnostics_topo.py:17: RuntimeWarning: invalid value encountered in divide
  InvBT = 1 / Laplacian


iteration i = 100; time step dt = 0.04461806359763545, ene = 3.275426000404675
42.938836097717285
qp = 155722.2789865699
Total time elapsed: 56.04820990562439 seconds


In [18]:
k = np.concatenate((np.arange(0, N//2 + 1), np.arange(-N//2 + 1, 0)))
dX = 1j * np.tile(k[None,:,None], (N, 1, 2))
dY = 1j * np.tile(k[:,None,None], (1, N, 2))

In [22]:
dX

array([[[ 0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+1.j],
        [ 0.+2.j,  0.+2.j],
        ...,
        [-0.-3.j, -0.-3.j],
        [-0.-2.j, -0.-2.j],
        [-0.-1.j, -0.-1.j]],

       [[ 0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+1.j],
        [ 0.+2.j,  0.+2.j],
        ...,
        [-0.-3.j, -0.-3.j],
        [-0.-2.j, -0.-2.j],
        [-0.-1.j, -0.-1.j]],

       [[ 0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+1.j],
        [ 0.+2.j,  0.+2.j],
        ...,
        [-0.-3.j, -0.-3.j],
        [-0.-2.j, -0.-2.j],
        [-0.-1.j, -0.-1.j]],

       ...,

       [[ 0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+1.j],
        [ 0.+2.j,  0.+2.j],
        ...,
        [-0.-3.j, -0.-3.j],
        [-0.-2.j, -0.-2.j],
        [-0.-1.j, -0.-1.j]],

       [[ 0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+1.j],
        [ 0.+2.j,  0.+2.j],
        ...,
        [-0.-3.j, -0.-3.j],
        [-0.-2.j, -0.-2.j],
        [-0.-1.j, -0.-1.j]],

       [[ 0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+1.j],
        [ 0.+2.j

In [ ]:
# Plotting
plt.figure()
plt.subplot(1, 2, 1)
plt.contour(xx, yy, qp[:, :, 0], 50)
plt.colorbar()
plt.title(f'barotropic mode at t = {t}')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(1, 2, 2)
plt.contour(xx, yy, qp[:, :, 1], 50)
plt.colorbar()
plt.title(f'baroclinic mode at t = {t}')
plt.xlabel('x')
plt.ylabel('y')



In [14]:
k[None,:].shape

(1, 256)

In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.loglog(np.arange(N // 2 + 1), np.mean(ke[:, -100:], axis=1), '.-', linewidth=1)
plt.title('kinetic energy spectrum')
plt.xlabel('wavenumber')

plt.subplot(1, 2, 2)
plt.loglog(np.arange(N // 2 + 1), np.mean(ape[:, -100:], axis=1), '.-', linewidth=1)
plt.title('potential energy spectrum')
plt.xlabel('wavenumber')



In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.loglog(np.arange(N // 2 + 1), np.mean(ene[:, -100:], axis=1), '.-', linewidth=1)
plt.title('energy spectrum')
plt.xlabel('wavenumber')

plt.subplot(1, 2, 2)
plt.loglog(np.arange(N // 2 + 1), np.mean(etp[:, -100:], axis=1), '.-', linewidth=1)
plt.title('enstrophy spectrum')
plt.xlabel('wavenumber')



In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.plot(np.arange(1, ene_t.shape[1] + 1), ene_t, '.-', linewidth=1)
plt.title('energy series')
plt.xlabel('time')

plt.subplot(1, 2, 2)
plt.plot(np.arange(1, etp_t.shape[1] + 1), etp_t, '.-', linewidth=1)
plt.title('enstrop series')
plt.xlabel('time')

plt.show()